__This notebook is not the final used version__. 
The final used version can be found in the __vref_walkability_index repository.__
__Notebook 02_PV_02b_Population (voronoi_points_within_aoi()).ipynb__

Notebook 02_PV_02b_Population is based on 02_PV_02a_Project_network_Voronois, which is based on this notebook.

# 11a_project_network_voronois [OUTDATED]

This notebook loads pop data (Looking for population and density) to nodes for the project.
Most of this notebook is based on Script 22-censo_pop_voronoi.py as of 2025/03/04.

__Inputs:__
* Area of interest (City, Guadalajara)
* Blocks with already pre-calculated pop data (from Script 22) pobcenso_inegi_20_mzaageb_mza
* Nodes from the project network, which was generated using the networks join procedure (Tessellations + Boeing)
  
__Outputs:__
* Voronoi polygons for the project_network
* Nodes with pop data (Population and density)

## Import libraries

In [1]:
first_folder_path = "../../../"

In [24]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Time processes
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Notebook config

In [27]:
# Data required in Script22-censo_pop_voronoi.py
city = 'Guadalajara'
year = '2020'

# ----- ----- ----- Projection to be used when needed ----- ----- ----- 
projected_crs = "EPSG:32613" #Guadalajara = 'EPSG:32613'

# ----- ----- ----- Output ----- ----- ----- 
output_dir = first_folder_path + f"data/external/volvo/project_network_voronoi/"
local_save = True

## Load data

### __Load data__ - Area of interest (Guadalajara)

In [17]:
# Database information
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020'

# Load
metro_query = f"SELECT * FROM {metro_schema}.{metro_table} WHERE \"city\" LIKE \'{city}\'"
metro_gdf = aup.gdf_from_query(pop_query, geometry_col='geometry')

city_gdf = metro_gdf.set_crs("EPSG:4326")
aoi = city_gdf.dissolve()

### __Load data__ - Pop data (Previously processed blocks generated on 22-censo_pop_voronoi.py for Guadalajara)

In [18]:
# Database information
pop_schema = 'censo'
pop_table = 'pobcenso_inegi_20_mzaageb_mza'

# Load
pop_query = f"SELECT * FROM {pop_schema}.{pop_table} WHERE \"city\" LIKE \'{city}\'"
pop_mza_gdf_calc = aup.gdf_from_query(pop_query, geometry_col='geometry')

# Show
print(pop_mza_gdf_calc.crs)
print(pop_mza_gdf_calc.dtypes)
print(pop_mza_gdf_calc.shape)
pop_mza_gdf_calc.head(2)

EPSG:4326
nom_ent         object
nom_mun         object
nom_loc         object
pobtot           int64
pobfem         float64
                ...   
cvegeo_mza      object
ambito          object
tipomza         object
geometry      geometry
city            object
Length: 238, dtype: object
(54804, 238)


,nom_ent,nom_mun,nom_loc,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,p_3ymas,...,cve_ageb,cve_mza,cvegeo_mun,cvegeo_loc,cvegeo_ageb,cvegeo_mza,ambito,tipomza,geometry,city
0,Jalisco,Zapopan,La Cuchilla (Extramuros de Tesistán),16,8.0,8.0,0.0,0.0,0.0,16.0,...,0299,011,14120,141200062,1412000620299,1412000620299011,Urbana,Típica,"POLYGON ((-103.46319 20.77136, -103.46369 20.7...",Guadalajara
1,Jalisco,Zapopan,La Cuchilla (Extramuros de Tesistán),18,8.0,10.0,0.0,0.0,0.0,8.0,...,0299,032,14120,141200062,1412000620299,1412000620299032,Urbana,Típica,"POLYGON ((-103.46409 20.77334, -103.46367 20.7...",Guadalajara


### __Load data__ - Network

In [8]:
network_dir = first_folder_path + "data/external/guadalajara_project_network/"

In [13]:
# Load nodes
nodes = gpd.read_file(network_dir + "guadalajara_nodes_proj_net_final.gpkg")
nodes = nodes_gdl.to_crs("EPSG:4326")

# Show
print(nodes.crs)
print(nodes.dtypes)
print(nodes.shape)
nodes.head(2)

EPSG:4326
osmid               int64
ntw_origin         object
streets_count       int64
x                 float64
y                 float64
geometry         geometry
dtype: object
(138597, 6)


,osmid,ntw_origin,streets_count,x,y,geometry
0,67637870229114485,ntw_01,3,676378.709485,2.291145e+06,POINT (-103.30631 20.71153)
1,67531453228992881,ntw_01,3,675314.533972,2.289929e+06,POINT (-103.31665 20.70065)


In [11]:
# Load edges
#edges_gdl = gpd.read_file(network_dir + "guadalajara_edges_proj_net_final.gpkg")
#edges_gdl = edges_gdl.to_crs("EPSG:4326")

# Show
#print(edges_gdl.crs)
#print(edges_gdl.dtypes)
#print(edges_gdl.shape)
#edges_gdl.head(2)

## Create voronoi polygons using nodes [Takes about +4000s, +1hr]

In [19]:
print("--- Creating voronois with nodes osmid data.")
# Create voronois
voronois_gdf = aup.voronoi_points_within_aoi(aoi,nodes,'osmid',projected_crs=projected_crs)
voronois_gdf = voronois_gdf[['osmid','geometry']]

# Show
print(voronois_gdf.shape)
voronois_gdf.head(2)

--- Creating voronois with nodes osmid data.
Error = -0.0%. Admissible.
(123827, 2)


,osmid,geometry
0,65612775227546601,"MULTIPOLYGON (((-103.50246 20.57129, -103.5024..."
1,66939976230232550,"MULTIPOLYGON (((-103.37272 20.81077, -103.3728..."


## Save voronoi polygons

In [28]:
if local_save:
    voronois_gdf.to_file(output_dir+"voronois_gdf.gpkg")

## Spatial intersection of voronoi polygons with blocks

In [20]:
# ------------------- (Finds area_pct, used to distribute pop data)
print("--- Creating spatial join between voronoi polygons and blocks.")
# Calculate total block area
pop_mza_gdf_calc = pop_mza_gdf_calc.to_crs(projected_crs)
pop_mza_gdf_calc['area_mza'] = pop_mza_gdf_calc.geometry.area
pop_mza_gdf_calc = pop_mza_gdf_calc.to_crs("EPSG:4326")
# Overlay blocks with voronoi
# (Spatial intersection, creates split blocks with data from the original block 
#  and the voronoi poly it falls in)
mza_voronoi = gpd.overlay(df1=pop_mza_gdf_calc, df2=voronois_gdf, how="intersection")
print("--- Calculating area_pct that corresponds to each osmid within each block.")
# Calculate pct of area of each block that falls in any given osmid voronoi polygon
mza_voronoi = mza_voronoi.to_crs("EPSG:6372")
mza_voronoi['area_voronoi'] = mza_voronoi.geometry.area
mza_voronoi = mza_voronoi.to_crs("EPSG:4326")
mza_voronoi['area_pct'] = mza_voronoi['area_voronoi']/mza_voronoi['area_mza']
# Drop used columns
mza_voronoi.drop(columns=['area_mza','area_voronoi'],inplace=True)

# Show
print(mza_voronoi.shape)
mza_voronoi.head(2)

--- Creating spatial join between voronoi polygons and blocks.
--- Calculating area_pct that corresponds to each osmid within each block.
(336724, 240)


,nom_ent,nom_mun,nom_loc,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,p_3ymas,...,cvegeo_mun,cvegeo_loc,cvegeo_ageb,cvegeo_mza,ambito,tipomza,city,osmid,geometry,area_pct
0,Jalisco,Zapopan,La Cuchilla (Extramuros de Tesistán),16,8.0,8.0,0.0,0.0,0.0,16.0,...,14120,141200062,1412000620299,1412000620299011,Urbana,Típica,Guadalajara,65997850229766921,"POLYGON ((-103.46318 20.77193, -103.46319 20.7...",0.226193
1,Jalisco,Zapopan,La Cuchilla (Extramuros de Tesistán),16,8.0,8.0,0.0,0.0,0.0,16.0,...,14120,141200062,1412000620299,1412000620299011,Urbana,Típica,Guadalajara,65997743229760294,"POLYGON ((-103.46319 20.77136, -103.46345 20.7...",0.144230


## Group pop data that corresponds to each node

In [22]:
# ------------------- (Groups mza_voronoi data by osmid)
print("--- Adding pop data by node.")
# List is similar to columns_of_interest inside function calculate_censo_nan_values_v1
columns_of_interest = ['pobtot','pobfem','pobmas',
                       'p_0a2','p_0a2_f','p_0a2_m',
                       'p_3a5','p_3a5_f','p_3a5_m',
                       'p_6a11','p_6a11_f','p_6a11_m',
                       'p_12a14','p_12a14_f','p_12a14_m',
                       'p_15a17','p_15a17_f','p_15a17_m',
                       'p_18a24','p_18a24_f','p_18a24_m',
                       'p_60ymas','p_60ymas_f','p_60ymas_m',
                       'p_3ymas','p_3ymas_f','p_3ymas_m',
                       'p_12ymas','p_12ymas_f','p_12ymas_m',
                       'p_15ymas','p_15ymas_f','p_15ymas_m',
                       'p_18ymas','p_18ymas_f','p_18ymas_m',
                       'pob0_14','pob15_64','pob65_mas']
if year == "2010":
    columns_of_interest.append('pcon_lim')
elif year == "2020":
    columns_of_interest.append('pcon_disc')

# Create pop_nodes_gdf (Will store nodes pop output by node) from nodes gdf.
pop_nodes_gdf = nodes[['osmid','geometry']].copy()

# For each column, sum pop data by osmid (Distributing pop data by considering pct of area of original block) and assigning it to its node
for col in columns_of_interest:
    # Turn column to numeric
    mza_voronoi[col] = pd.to_numeric(mza_voronoi[col])
    # Calculate pop data proportionaly to pct of overlayed voronoi area relative to block area
    mza_voronoi[f'voronoi_{col}'] = mza_voronoi[col] * mza_voronoi['area_pct']
    # Group data by osmid
    osmid_grouped_data = mza_voronoi.groupby('osmid').agg({f'voronoi_{col}':np.sum})
    # Merge data to nodes_gdf
    osmid_grouped_data.reset_index(inplace=True)
    pop_nodes_gdf = pd.merge(pop_nodes_gdf, osmid_grouped_data, on='osmid')
    pop_nodes_gdf.rename(columns={f'voronoi_{col}':col},inplace=True)
print(f"--- Distributed block data to nodes, total population of {pop_nodes_gdf['pobtot'].sum()} for area of interest.")

--- Adding pop data by node.
--- Distributed block data to nodes, total population of 4970593.798612511 for area of interest.


In [26]:
# Show
print(pop_nodes_gdf.shape)
pop_nodes_gdf.head(2)

(123827, 43)


,osmid,geometry,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,p_3a5,p_3a5_f,...,p_15ymas_f,p_15ymas_m,p_18ymas,p_18ymas_f,p_18ymas_m,pob0_14,pob15_64,pob65_mas,pcon_disc,dens_pob_ha
0,67637870229114485,POINT (-103.30631 20.71153),74.258051,39.142375,35.115676,4.018620,2.771184,1.247437,3.518030,2.158454,...,29.709683,28.169779,54.974162,28.103850,26.870312,16.378589,51.800630,6.078832,2.038236,200.003244
1,67531453228992881,POINT (-103.31665 20.70065),178.698417,94.179297,84.519120,4.926541,3.027571,1.898969,6.608257,2.880036,...,75.416105,68.066850,133.803101,70.732529,63.070571,35.215463,118.211579,25.271376,5.402879,318.299935


## Calculate pop density in nodes (Using it's voronoi polygon's area)

In [23]:
# Calculate whole voronoi's area
voronois_gdf = voronois_gdf.to_crs("EPSG:6372")
voronois_gdf['area_has'] = voronois_gdf.area/10000
voronois_gdf = voronois_gdf.to_crs("EPSG:4326")
# Merge poptot data by node with the whole voronoi polygon using 'osmid'
dens_voronoi = pd.merge(pop_nodes_gdf[['osmid','pobtot']], voronois_gdf[['osmid','area_has']], on='osmid')
# Calculate density
dens_voronoi['dens_pob_ha'] = dens_voronoi['pobtot'] / dens_voronoi['area_has']
# Merge back that density data to nodes_gdf using 'osmid'
pop_nodes_gdf = pd.merge(pop_nodes_gdf, dens_voronoi[['osmid','dens_pob_ha']], on='osmid')
print(f"--- Added density to nodes using each voronoi polygon's area.")

--- Added density to nodes using each voronoi polygon's area.


In [25]:
# Show
print(pop_nodes_gdf.shape)
pop_nodes_gdf.head(2)

(123827, 43)


,osmid,geometry,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,p_3a5,p_3a5_f,...,p_15ymas_f,p_15ymas_m,p_18ymas,p_18ymas_f,p_18ymas_m,pob0_14,pob15_64,pob65_mas,pcon_disc,dens_pob_ha
0,67637870229114485,POINT (-103.30631 20.71153),74.258051,39.142375,35.115676,4.018620,2.771184,1.247437,3.518030,2.158454,...,29.709683,28.169779,54.974162,28.103850,26.870312,16.378589,51.800630,6.078832,2.038236,200.003244
1,67531453228992881,POINT (-103.31665 20.70065),178.698417,94.179297,84.519120,4.926541,3.027571,1.898969,6.608257,2.880036,...,75.416105,68.066850,133.803101,70.732529,63.070571,35.215463,118.211579,25.271376,5.402879,318.299935


## Save nodes with pop data

In [29]:
if local_save:
    pop_nodes_gdf.to_file(output_dir+"pop_nodes_gdf.gpkg")